In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

C:\Users\ayesha\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 6s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/10
1719/1719 - 3s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/10
1719/1719 - 3s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/10
1719/1719 - 3s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/10
1719/1719 - 3s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006
Epoch 6/10
1719/1719 - 3s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052
Epoch 7/10
1719/1719 - 3s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102
Epoch 8/10
1719/1719 - 3s - loss: 0.3485 - accuracy: 0.9022 - val_loss: 0.3209 - val_accuracy: 0.9138
Epoch 9/10
1719/1719 - 3s - loss: 0.3356 - accuracy: 0.9053 - val_loss: 0.3111 - val_accuracy: 0.9152
Epoch 10/10
1719/1719 - 3s - loss: 0.3251 - accuracy: 0.9077 - val_loss: 0.3016 - 

In [7]:
model.save("full_mnist_model.h5")

In [8]:
load_full_mnist_model = tf.keras.models.load_model("full_mnist_model.h5")

In [9]:
load_full_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [10]:
# Check trainable or not:

for layer in load_full_mnist_model.layers:
    print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


In [13]:
for layer in load_full_mnist_model.layers[:-1]:
    layer.trainable = False
    print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [14]:
for layer in load_full_mnist_model.layers:
    print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [15]:
lower_pretrained_models = load_full_mnist_model.layers[:-1]
new_model = tf.keras.models.Sequential(lower_pretrained_models)
new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [16]:
#only last layer 202 paramters are trainable
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 265,802
Trainable params: 202
Non-trainable params: 265,600
______________________________________________

In [17]:
def update_even_odd_labels(labels):
    for idx, label in enumerate(labels):
        labels[idx] = np.where(label % 2 == 0, 1 , 0)
    return labels

In [18]:
y_train_bin, y_valid_bin, y_test_bin = update_even_odd_labels([y_train, y_valid, y_test])

In [26]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [24]:
X_train.shape, y_train_bin.shape

((55000, 28, 28), (55000,))

In [25]:
X_valid.shape, y_valid_bin.shape

((5000, 28, 28), (5000,))

In [27]:
new_history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)
#loss here starts from 0.8288 as the model hass already trained for lower layers

Epoch 1/10
1719/1719 - 3s - loss: 0.3898 - accuracy: 0.8288 - val_loss: 0.3247 - val_accuracy: 0.8676
Epoch 2/10
1719/1719 - 2s - loss: 0.3300 - accuracy: 0.8602 - val_loss: 0.3049 - val_accuracy: 0.8754
Epoch 3/10
1719/1719 - 2s - loss: 0.3163 - accuracy: 0.8660 - val_loss: 0.2948 - val_accuracy: 0.8796
Epoch 4/10
1719/1719 - 2s - loss: 0.3083 - accuracy: 0.8701 - val_loss: 0.2885 - val_accuracy: 0.8832
Epoch 5/10
1719/1719 - 2s - loss: 0.3023 - accuracy: 0.8725 - val_loss: 0.2834 - val_accuracy: 0.8846
Epoch 6/10
1719/1719 - 2s - loss: 0.2978 - accuracy: 0.8752 - val_loss: 0.2792 - val_accuracy: 0.8874
Epoch 7/10
1719/1719 - 2s - loss: 0.2939 - accuracy: 0.8772 - val_loss: 0.2758 - val_accuracy: 0.8872
Epoch 8/10
1719/1719 - 2s - loss: 0.2907 - accuracy: 0.8788 - val_loss: 0.2728 - val_accuracy: 0.8890
Epoch 9/10
1719/1719 - 2s - loss: 0.2876 - accuracy: 0.8797 - val_loss: 0.2708 - val_accuracy: 0.8906
Epoch 10/10
1719/1719 - 2s - loss: 0.2851 - accuracy: 0.8816 - val_loss: 0.2678 - 

In [28]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 1ms/step - loss: 0.2768 - accuracy: 0.8861


[0.2768484950065613, 0.8860999941825867]

In [29]:
X_new = X_test[:3]

In [30]:
y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [31]:
np.argmax(new_model.predict(X_new), axis = -1)

array([0, 1, 0], dtype=int64)

## Batch Norm Apporach One

In [32]:
del model

In [42]:
LAYERS_BN = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN)

In [36]:
(3136 + 1200 + 400)/2 + 266610

268978.0

In [34]:
#784 * 4 = 3136 (4 for 4 parameters mean, variance, gamma, beta)
#similarly 300 *4 = 1200 & 100 *4 = 400
#so total traniable parameter = (3136 + 1200 + 400)/2 + 266,610 (divinding by 2 bcoz only gamma & beta are learnable parameters)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_4 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_5 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_6 (Dense)              (None, 10)               

In [37]:
bn1 = model.layers[1]

In [38]:
bn1.variables

[<tf.Variable 'batch_normalization/gamma:0' shape=(784,) dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1

In [39]:
for variable in bn1.variables:
    print(variable.name, variable.trainable)

batch_normalization/gamma:0 True
batch_normalization/beta:0 True
batch_normalization/moving_mean:0 False
batch_normalization/moving_variance:0 False


In [40]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

C:\Users\ayesha\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [41]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 10s - loss: 0.8370 - accuracy: 0.7443 - val_loss: 0.4334 - val_accuracy: 0.8766
Epoch 2/10
1719/1719 - 7s - loss: 0.4238 - accuracy: 0.8781 - val_loss: 0.3158 - val_accuracy: 0.9106
Epoch 3/10
1719/1719 - 6s - loss: 0.3470 - accuracy: 0.9017 - val_loss: 0.2651 - val_accuracy: 0.9246
Epoch 4/10
1719/1719 - 6s - loss: 0.3031 - accuracy: 0.9111 - val_loss: 0.2375 - val_accuracy: 0.9318
Epoch 5/10
1719/1719 - 7s - loss: 0.2744 - accuracy: 0.9214 - val_loss: 0.2155 - val_accuracy: 0.9380
Epoch 6/10
1719/1719 - 6s - loss: 0.2554 - accuracy: 0.9256 - val_loss: 0.2004 - val_accuracy: 0.9416
Epoch 7/10
1719/1719 - 6s - loss: 0.2376 - accuracy: 0.9306 - val_loss: 0.1890 - val_accuracy: 0.9458
Epoch 8/10
1719/1719 - 6s - loss: 0.2233 - accuracy: 0.9363 - val_loss: 0.1796 - val_accuracy: 0.9476
Epoch 9/10
1719/1719 - 7s - loss: 0.2109 - accuracy: 0.9385 - val_loss: 0.1732 - val_accuracy: 0.9502
Epoch 10/10
1719/1719 - 6s - loss: 0.2017 - accuracy: 0.9411 - val_loss: 0.1645 -

## Batch Norm Apporach Two

In [43]:
del model

In [44]:
LAYERS_BN_BIAS_FALSE = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN_BIAS_FALSE)

In [46]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [47]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 13s 7ms/step - loss: 1.1456 - accuracy: 0.6719 - val_loss: 0.6169 - val_accuracy: 0.8438
Epoch 2/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5757 - accuracy: 0.8537 - val_loss: 0.4349 - val_accuracy: 0.8868
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4570 - accuracy: 0.8805 - val_loss: 0.3577 - val_accuracy: 0.9054
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3943 - accuracy: 0.8952 - val_loss: 0.3148 - val_accuracy: 0.9154
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3539 - accuracy: 0.9039 - val_loss: 0.2858 - val_accuracy: 0.9222
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3269 - accuracy: 0.9095 - val_loss: 0.2640 - val_accuracy: 0.9258
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3049 - accuracy: 0.9154 - val_loss: 0.2474 - val_accurac